[![pythonista](img/pythonista.png)](https://www.pythonista.io)

# La especificación *JSON Schema*.

### Desventajas de *JSON*.

A diferencia de *XML* el formato *JSON* sólo cuenta con con una definición muy limitada de tipos de datos. Debido a esto no es posible realizar validaciones avanzadas de los datos.

## Modelos y esquemas (*schemas*).

### Estado de un objeto.

En la programación orientada a objetos, es posible modelar de forma simple las características de objetos en un momento dado. A esto se le conoce como el estado de un objeto.

Cada dato que describe el estado de un objeto es ligado a un nombre o identificador al cual se le conoce como propiedad o atributo.

El modelo base de los objetos se conoce como *clase* y a los objetos emanados de dicha clase se les conoce como instancias.

In [ ]:
from dataclasses import dataclass, asdict

In [ ]:
@dataclass
class Alumno():
    nombre: str
    primer_apellido: str
    promedio: float
    segundo_apellido: str = ""
    inscrito: bool = True

In [ ]:
alumnonuevo = Alumno(nombre="Juan",
                     primer_apellido="Pérez", 
                     promedio=8.6)

In [ ]:
alumnonuevo

### Serialización de objetos.

Para poder transmitir el estado de un objeto es necesario extraerlo y almacenarlo en un formato capaz de ser enviado y reconstruido sin pérdidas.

In [ ]:
asdict(alumnonuevo)

## La especificación *JSON Schema*.

En función de la necesidad de crear una especificación que permita extender a *JSON* para representar estructuras de datos más complejas, se ha publicado [*JSON Schema*](https://json-schema.org/).

In [2]:
!pip install jsonschema


[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import jsonschema

### Tipos básicos.

De forma similar a las versiones antiguas de *Javascript* previas a *ECMAScript 2016*, *JSON Schema* define los siguientes tipos de datos.  

https://json-schema.org/understanding-json-schema/reference/type.html

* [```string```](https://json-schema.org/understanding-json-schema/reference/string.html#string), los cuales corresponden a cadenas de caracteres y cuenta con [formatos predeterminados](https://json-schema.org/understanding-json-schema/reference/string.html#format).
* [```number```](https://json-schema.org/understanding-json-schema/reference/numeric.html#number), los cuales corresponden a tipos de punto flotante.
* [```integer```](https://json-schema.org/understanding-json-schema/reference/numeric.html#integer), los cuales corresponden a números enteros.
* [```boolean```](https://json-schema.org/understanding-json-schema/reference/boolean.html#boolean), los cuales corrersponden a valores booleanos.
* [```null```](https://json-schema.org/understanding-json-schema/reference/null.html), el cual corresponden al valor nulo.
* [```array```](https://json-schema.org/understanding-json-schema/reference/array.html), los cuales corresponden a arreglos de *Javascript*.

### El tipo ```object```.

El tipo [```object```](https://json-schema.org/understanding-json-schema/reference/object.html) permite crear estructuras similares a los objetos de *Javascript* mediante la definición de [propiedades](https://json-schema.org/understanding-json-schema/reference/object.html#properties).

**Ejemplo:**

* La siguiente celda creará el esquema ```esquema_alumno```.

In [2]:
esquema_alumno = {
    "type": "object",
    "properties": {
        "Nombre": {"type": "string",
                   "minLength":1,
                   },
        "Primer Apellido": {"type": "string", 
                   "minLength":1,},
        "Segundo Apellido": {"type": "string", 
                   "minLength":1,},
        "Carrera": {"type": "string"},
        "Semestre": {"type": "number",
                   "minimum": 1,
                   "maximum": 50,},
        "Promedio": {"type": "number",
                   "minimum": 0,
                   "maximum": 10,},
        "Al Corriente": {"type": "boolean"},
    },
    "required": ["Nombre",  "Primer Apellido", "Carrera", "Semestre",
                 "Promedio", "Al Corriente"],
            "additionalProperties": False,
}

## La función ```jsonschema.validate()```.


```
jsonschema.validate(<estructura>, <esquema>)
```

In [25]:
jsonschema.validate({'Al Corriente': False,
                     'Carrera': 'Arquitectura',
                     'Nombre': 'Pedro', 
                     'Primer Apellido': 'Solis', 
                     'Promedio': 7.8, 
                     'Semestre': 5,}, 
                    esquema_alumno)

In [26]:
jsonschema.validate({'Al Corriente': False,
                     'Carrera': 'Arquitectura',
                     'Nombre': 'Pedro', 
                     'Primer Apellido': 'Solis', 
                     'Promedio': 7.8, 
                     'Semestre': 5,
                     'Género': 'F',},
                    esquema_alumno)

ValidationError: Additional properties are not allowed ('Género' was unexpected)

Failed validating 'additionalProperties' in schema:
    {'additionalProperties': False,
     'properties': {'Al Corriente': {'type': 'boolean'},
                    'Carrera': {'type': 'string'},
                    'Nombre': {'minLength': 1, 'type': 'string'},
                    'Primer Apellido': {'minLength': 1, 'type': 'string'},
                    'Promedio': {'maximum': 10,
                                 'minimum': 0,
                                 'type': 'number'},
                    'Segundo Apellido': {'minLength': 1, 'type': 'string'},
                    'Semestre': {'maximum': 50,
                                 'minimum': 1,
                                 'type': 'number'}},
     'required': ['Nombre',
                  'Primer Apellido',
                  'Carrera',
                  'Semestre',
                  'Promedio',
                  'Al Corriente'],
     'type': 'object'

In [27]:
jsonschema.validate({'Al Corriente': False, 
          'Carrera': 'Arquitectura', 
          'Nombre': 'Pedro', 
          'Primer Apellido': 'Solis', 
          'Promedio': 7.8, 
          'Semestre': -7,}, 
                    esquema_alumno)

ValidationError: -7 is less than the minimum of 1

Failed validating 'minimum' in schema['properties']['Semestre']:
    {'maximum': 50, 'minimum': 1, 'type': 'number'}

On instance['Semestre']:
    -7

### Enumeradores.

https://cswr.github.io/JsonSchema/spec/generic_keywords/#enumerated-values

In [28]:
esquema_alumno = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "Nombre": {"type": "string",
                   "minLength":1,
                   },
        "Primer Apellido": {"type": "string", 
                   "minLength":1,},
        "Segundo Apellido": {"type": "string", 
                   "minLength":1,},
        "Carrera": {"type": "string",
                    "enum": ["Sistemas", 
                             "Derecho", 
                             "Medicina", 
                             "Actuaría"],
                   },
        "Semestre": {"type": "number",
                   "minimum": 1,
                   "maximum": 50,},
        "Promedio": {"type": "number",
                   "minimum": 0,
                   "maximum": 10,},
        "Al Corriente": {"type": "boolean"},
    },
    "required": ["Nombre",  "Primer Apellido", "Carrera", "Semestre",
                 "Promedio", "Al Corriente"]
}

In [29]:
jsonschema.validate({'Al Corriente': False,
                     'Carrera': 'Arquitectura',
                     'Nombre': 'Pedro', 
                     'Primer Apellido': 'Solis', 
                     'Promedio': 7.8, 
                     'Semestre': 5,
                     'Género': 'F',},
                    esquema_alumno)

ValidationError: 'Arquitectura' is not one of ['Sistemas', 'Derecho', 'Medicina', 'Actuaría']

Failed validating 'enum' in schema['properties']['Carrera']:
    {'enum': ['Sistemas', 'Derecho', 'Medicina', 'Actuaría'],
     'type': 'string'}

On instance['Carrera']:
    'Arquitectura'

### Composición.

https://json-schema.org/understanding-json-schema/reference/combining.html#combining

* [```allOf```](https://json-schema.org/understanding-json-schema/reference/combining.html#allof)
* [```anyOf```](https://json-schema.org/understanding-json-schema/reference/combining.html#anyof)
* [```oneOf```](https://json-schema.org/understanding-json-schema/reference/combining.html#oneof)
* [```not```](https://json-schema.org/understanding-json-schema/reference/combining.html#not)

## Implementaciones de *JSON Schema*.

https://json-schema.org/implementations.html

## Exportación del esquema.

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2022.</p>